In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
html = list(soup.children)[2]
body = list(html.children)[3]
p = list(body.children)[1]

In [8]:
p.get_text()

'Here is some simple content for this page.'

In [9]:
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]

In [10]:
tonight.prettify()

'<div class="tombstone-container">\n <p class="period-name">\n  Overnight\n  <br/>\n  <br/>\n </p>\n <p>\n  <img alt="Overnight: Mostly cloudy, with a low around 56. West southwest wind around 10 mph. " class="forecast-icon" src="newimages/medium/nbkn.png" title="Overnight: Mostly cloudy, with a low around 56. West southwest wind around 10 mph. "/>\n </p>\n <p class="short-desc">\n  Mostly Cloudy\n </p>\n <p class="temp temp-low">\n  Low: 56 °F\n </p>\n</div>'

In [11]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Overnight
Mostly Cloudy
Low: 56 °F


In [13]:
img = tonight.find("img")
desc = img['title']
print(desc)

Overnight: Mostly cloudy, with a low around 56. West southwest wind around 10 mph. 


In [14]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Overnight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight',
 'Monday',
 'MondayNight']

## Extracting all the information from the page

In [19]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Mostly Cloudy', 'Mostly Cloudy', 'ChanceShowers', 'ChanceShowers', 'Mostly Cloudy', 'Mostly Cloudy', 'Mostly Cloudy', 'Partly Sunny', 'Partly Cloudy']
['Low: 56 °F', 'High: 69 °F', 'Low: 59 °F', 'High: 70 °F', 'Low: 56 °F', 'High: 68 °F', 'Low: 56 °F', 'High: 67 °F', 'Low: 55 °F']
['Overnight: Mostly cloudy, with a low around 56. West southwest wind around 10 mph. ', 'Friday: Mostly cloudy, with a high near 69. Southwest wind 11 to 20 mph, with gusts as high as 30 mph. ', 'Friday Night: A 40 percent chance of showers after 11pm.  Increasing clouds, with a low around 59. South wind 10 to 16 mph, with gusts as high as 26 mph.  New precipitation amounts of less than a tenth of an inch possible. ', 'Saturday: A 50 percent chance of showers.  Partly sunny, with a high near 70. South southwest wind 8 to 15 mph, with gusts as high as 18 mph.  New precipitation amounts of less than a tenth of an inch possible. ', 'Saturday Night: Mostly cloudy, with a low around 56. South southwest wind 6 to

## Combining our data into a Pandas Dataframe

In [20]:
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,period,short_desc,temp,desc
0,Overnight,Mostly Cloudy,Low: 56 °F,"Overnight: Mostly cloudy, with a low around 56..."
1,Friday,Mostly Cloudy,High: 69 °F,"Friday: Mostly cloudy, with a high near 69. So..."
2,FridayNight,ChanceShowers,Low: 59 °F,Friday Night: A 40 percent chance of showers a...
3,Saturday,ChanceShowers,High: 70 °F,Saturday: A 50 percent chance of showers. Par...
4,SaturdayNight,Mostly Cloudy,Low: 56 °F,"Saturday Night: Mostly cloudy, with a low arou..."
5,Sunday,Mostly Cloudy,High: 68 °F,"Sunday: Mostly cloudy, with a high near 68."
6,SundayNight,Mostly Cloudy,Low: 56 °F,"Sunday Night: Mostly cloudy, with a low around..."
7,Monday,Partly Sunny,High: 67 °F,"Monday: Partly sunny, with a high near 67."
8,MondayNight,Partly Cloudy,Low: 55 °F,"Monday Night: Partly cloudy, with a low around..."


In [21]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    56
1    69
2    59
3    70
4    56
5    68
6    56
7    67
8    55
Name: temp_num, dtype: object

In [22]:
weather["temp_num"].mean()

61.77777777777778

In [23]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool